# Alternative PygmalionAI notebook for Google Colab

Version: GPU

This notebook is an unofficial method to run [PygmalionAI LLM models](https://github.com/PygmalionAI) that uses **oobabooga**'s [text-generation-webui project](https://github.com/oobabooga/text-generation-webui).

Run the cells below in order to set it up.

---

After completing step 3 you may upload your character definitions in `.json` format to [Google Drive](https://drive.google.com/) by navigating to the following directory:
  * **My Drive** &rarr; **Colab-TextGen** &rarr; **characters**

At the end of step 5 click on the `.gradio.live` link to access the web interface.

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

# From: https://github.com/henk717/KoboldAI

%%html
<b>Press play on the music player to keep the tab alive (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title 2. Prepare the Conda environment

%cd /tmp
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
%env PYTHONPATH=
!which conda 2>/dev/null || bash Miniconda3-latest-Linux-x86_64.sh -b -p /usr/local -f

%env CI=true
!conda update -y -q -c defaults --all
!conda install -y -q -c defaults conda python
!conda install -y -q -c pytorch -c nvidia pytorch torchvision torchaudio pytorch-cuda=11.7
!conda clean -y -q --all

In [ ]:
#@title 3. Clone the repository into your Google Drive and install dependencies

from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive
!git -C /content/drive/MyDrive/Colab-TextGen pull || git clone https://github.com/oobabooga/text-generation-webui /content/drive/MyDrive/Colab-TextGen
%cd /content/drive/MyDrive/Colab-TextGen
!pip install --no-cache-dir --progress-bar=off --root-user-action=ignore -r requirements.txt bitsandbytes

In [ ]:
#@title 4. Select and download a PygmalionAI model

Model = "Pygmalion 6B" #@param ["Pygmalion 6B", "Pygmalion 1.3B"] {allow-input: true}

if Model == "Pygmalion 6B":
  huggingface_org = "PygmalionAI"
  chosen_model = "pygmalion-6b"
elif Model == "Pygmalion 1.3B":
  huggingface_org = "PygmalionAI"
  chosen_model = "pygmalion-1.3b"

%cd /content/drive/MyDrive/Colab-TextGen
!mkdir -p /content/models/$chosen_model
!ln -s -f /content/models/$chosen_model/ models/$chosen_model
![[ ! -f models/$chosen_model/config.json ]] && python download-model.py $huggingface_org/$chosen_model

In [ ]:
#@title 5. Launch the web interface

!cd /content/drive/MyDrive/Colab-TextGen
!python server.py --cai-chat --share --auto-devices --load-in-8bit --model $chosen_model